In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import time

from multiprocessing import Pool

import pandas as pd

In [2]:
# LOAD PROFILE URLS FROM FILE
urls = [line.rstrip('\n') for line in open("linkedin_profile_urls.txt")]
print("Number of urls: ", len(urls))

Number of urls:  902


In [3]:
# REMOVE DUPLICATE URLS
urls = list(dict.fromkeys(urls))
print("Number of urls: ", len(urls))

Number of urls:  590


In [21]:
all_name = []
all_headline = []
all_location = []
all_no_of_connections = []
all_about = []
all_experiences = []
all_educations = []
all_l_and_c = []
all_skills = []

In [24]:
# FUNCTION TO SCRAPE
def scrape(url):
    # GO TO URL
    browser.get(url)
    
    # LOGIN
    try:
        sign_in_btn = browser.find_element_by_xpath("//p[@class='form-subtext login']/a")
        sign_in_btn.click()
        username = browser.find_element_by_xpath("//input[@class='login-email']")
        username.send_keys("knock_it_out@hotmail.com")
        password = browser.find_element_by_xpath("//input[@class='login-password']")
        password.send_keys("P@ssw0rd123")
        submit_btn = browser.find_element_by_xpath("//input[@class='login submit-button']")
        submit_btn.click()

        WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.ID, "experience-section")))
        time.sleep(3)
    except:
        pass
    
    # CLICK SHOW MORE
    end = False
    while(not end):
        try:
            show_more_btn = browser.find_elements_by_xpath("//button[@class='pv-profile-section__see-more-inline pv-profile-section__text-truncate-toggle link link-without-hover-state']")

            if(len(show_more_btn) == 0):
                end = True

            for btn in show_more_btn:
                btn.location_once_scrolled_into_view
                browser.execute_script("window.scrollTo(0, window.scrollY - 200)")
                btn.click()
        except:
            end = True

    # CLICK SEE MORE
    try:
        see_more_btn = browser.find_elements_by_xpath("//a[@class='lt-line-clamp__more']")

        for btn in see_more_btn:
            try:
                btn.location_once_scrolled_into_view
                browser.execute_script("window.scrollTo(0, window.scrollY - 200)")
                btn.click()
                time.sleep(0.1)
            except:
                pass   
    except:
        pass

    # CLICK SKILLS SHOW MORE
    try:
        skills_show_more_btn = browser.find_element_by_xpath("//button[@aria-controls='skill-categories-expanded']")
        skills_show_more_btn.location_once_scrolled_into_view
        browser.execute_script("window.scrollTo(0, window.scrollY - 200)")
        skills_show_more_btn.click()
    except:
        pass
    
    #----------------------COLLECTION----------------------------
    # GET NAME
    name = browser.find_element_by_xpath("//li[@class='inline t-24 t-black t-normal break-words']").text
    all_name.append(name)
    # GET HEADLINE
    try:
        headline = browser.find_element_by_xpath("//h2[@class='mt1 t-18 t-black t-normal']").text
    except:
        headline = "Nil"
    all_headline.append(headline)
    # GET LOCATION
    country = browser.find_element_by_xpath("//li[@class='t-16 t-black t-normal inline-block']").text
    all_location.append(country)
    # GET NUMBER OF CONNECTIONS
    no_of_connections = browser.find_element_by_xpath("//span[@class='t-16 t-black t-normal']").text
    all_no_of_connections.append(no_of_connections)
    # GET ABOUT
    try:
        about = browser.find_element_by_xpath("//span[@class='lt-line-clamp__raw-line']").text
    except:
        about = "Nil"
    if about == "Nil":
        try:
            about = browser.find_element_by_xpath("//span[@class='lt-line-clamp__line lt-line-clamp__line--last']").text
        except:
            about = "Nil"
    all_about.append(about)
    
    # GET LIST OF EXPERIENCE
    experience_list = []

    experience = browser.find_element_by_id("experience-section")
    positions = experience.find_elements_by_class_name("pv-position-entity")

    for position in positions:
        entries = position.find_elements_by_class_name("pv-entity__position-group-role-item")

        if(len(entries)>0):
            company = position.find_element_by_class_name("pv-entity__company-summary-info").find_element_by_tag_name("h3").text.replace("Company Name", "").strip()
            for entry in entries:
                title = entry.find_element_by_class_name("pv-entity__summary-info--background-section").find_element_by_tag_name("h3").text.replace("Title", "").strip()
                duration_date = entry.find_element_by_class_name("pv-entity__summary-info--background-section").find_element_by_class_name("pv-entity__date-range").text.replace("Dates Employed", "").strip()
                duration_length = entry.find_element_by_class_name("pv-entity__summary-info--background-section").find_elements_by_tag_name("h4")[1].text.replace("Employment Duration", "").strip()

                try:
                    description = entry.find_element_by_class_name("pv-entity__description").find_element_by_tag_name("span").text.strip()
                except:
                    description = 'Nil'

                experience_list.append({'title':title, 'company':company, 'duation_date':duration_date, 'duration_length':duration_length, 'description':description})

        else:
            title = position.find_element_by_tag_name("h3").text.strip()
            company = position.find_element_by_class_name("pv-entity__secondary-title").text.strip()
            duration_date = position.find_element_by_class_name("pv-entity__date-range").text.replace("Dates Employed", "").strip()
            duration_length = position.find_element_by_class_name("pv-entity__bullet-item-v2").text.replace("Employment Duration", "").strip()

            try:
                description = position.find_element_by_class_name("pv-entity__description").text.strip()
            except:
                description = 'Nil'

            experience_list.append({'title':title, 'company':company, 'duation_date':duration_date, 'duration_length':duration_length, 'description':description})
            
    all_experiences.append(experience_list)
    
    # GET LIST OF EDUCATION
    education_list = []
    try:
        educations = browser.find_element_by_id("education-section")

        for education in educations.find_elements_by_class_name("pv-education-entity"):
            school = education.find_element_by_class_name("pv-entity__school-name").text.strip()
            try:
                course_type = education.find_element_by_class_name("pv-entity__degree-name").text.replace("Degree Name", "").strip()
            except:
                course_type = "Nil"
            try:
                course = education.find_element_by_class_name("pv-entity__fos").text.replace("Field Of Study", "").strip()
            except:
                course = 'Nil'
            try:
                duration = education.find_element_by_class_name("pv-entity__dates").text.replace("Dates attended or expected graduation", "").strip()
            except:
                duration = "Nil"
            try:
                description = education.find_element_by_class_name("pv-entity__description").text.strip()
            except:
                description = "Nil"

            education_list.append({'school':school, 'type':course_type, 'course':course, 'duration':duration, 'description':description})

    except:
        education_list.append(None)
        
    all_educations.append(education_list)
    
    # GET LIST OF LICENSES AND CERTIFICATIONS
    l_and_c_list = []
    try:
        l_and_c = browser.find_element_by_id("certifications-section")

        for landc in l_and_c.find_elements_by_class_name("pv-certification-entity"):
            name = landc.find_element_by_class_name("pv-certifications__summary-info").find_element_by_tag_name("h3").text.strip()
            try:
                provider = landc.find_element_by_class_name("pv-certifications__summary-info").find_element_by_tag_name("p").text.replace("Issuing authority", "").strip()
            except:
                provider = "Nil"
            l_and_c_list.append({'name':name, 'provider':provider})
    except:
        l_and_c_list.append(None)
        
    all_l_and_c.append(l_and_c_list)
    
    # GET LIST OF SKILLS
    skills_list = []
    try:
        skills = browser.find_elements_by_xpath("//span[@class='pv-skill-category-entity__name-text t-16 t-black t-bold']")
        for skill in skills:
            skills_list.append(skill.text.strip())
    except:
        skills_list.append(None)
        
    all_skills.append(skills_list)
    
    # GET LIST OF OTHER PROFILES
    profiles = browser.find_elements_by_xpath("//a[@data-control-name='browsemap_profile']")
    f = open("extra_linkedin_profile_urls.txt", "a")
    for profile in profiles:
        f.write("%s\n" %profile.get_attribute("href"))
    f.close()

In [25]:
# OPEN CHROME BROWSER
browser = webdriver.Chrome("C:/chromedriver")
for url in urls[:5]:
    scrape(url)

In [26]:
profiles = pd.DataFrame({'name':all_name,
                         'headline':all_headline,
                         'location':all_location,
                         'no_of_connections':all_no_of_connections,
                         'about':all_about,
                         'experiences':all_experiences,
                         'educations':all_educations,
                         'licenses_and_certificates': all_l_and_c,
                         'skills': all_skills,})

In [27]:
profiles

,name,headline,location,no_of_connections,about,experiences,educations,licenses_and_certificates,skills
0,Nikita Gupta,Data Analyst at TBWA\The Disruption Company | ...,Singapore,500+ connections,"I am currently at TBWA,Singapore helping adver...","[{'title': 'Data Analyst', 'company': 'TBWA\ G...","[{'school': 'Singapore Management University',...",[{'name': 'DoubleClick Campaign Manager Fundam...,"[Databases, Business Analysis, Data Structures..."
1,Jayden Ho,Senior Data Analyst at Publicis Singapore,Singapore,339 connections,Uncovering insights and trends for the consume...,"[{'title': 'Senior Data Analyst, Strategy', 'c...","[{'school': 'RMIT University', 'type': 'Bachel...","[{'name': 'Intro to SQL for Data Science', 'pr...","[Business Analytics, Data Visualization, Socia..."
2,Timothy Ong,Senior Data Analyst at Shopee,Singapore,463 connections,Timothy has a huge passion for data analytics....,"[{'title': 'Senior Data Analyst', 'company': '...","[{'school': 'Singapore Management University',...","[{'name': 'Six Sigma Green Belt', 'provider': ...",[]
3,Zi Lin,Data Analyst at GovTech,Singapore,147 connections,• Review and develop standard operating proced...,"[{'title': 'Data Analyst, Data Science & Artif...","[{'school': 'Harvard University', 'type': 'Mas...","[{'name': 'Exploratory Data Analysis', 'provid...","[R, Investigation, Statistical Data Analysis, ..."
4,Minghao Liu,Data Analyst at Energy Aspects,Singapore,307 connections,• Investigate behaviors of users and advertise...,"[{'title': 'Data Analyst', 'company': 'Energy ...",[{'school': 'National University of Singapore'...,"[{'name': '计算导论与C语言基础', 'provider': 'Coursera'...","[SQL, R, Python, Data Analysis, Data Mining, M..."


In [29]:
all_educations[0]

[{'school': 'Singapore Management University',
  'type': 'Master of IT in Business - Analytics',
  'course': 'Nil',
  'duration': '2015 – 2016',
  'description': '• A+ Grade (4.3/4.0) for Internship Performance | Top 1%\n• SMU MBA Consulting Club – Member\n\nSingapore Management University is an elite national university in Singapore that specializes in business and management studies, ranked #2 in Asia and #34 Worldwide by The Financial Times.\nCurrently studying core courses of Analytics like Statistics,Technology Fundamentals for Analytics in which I will learn languages like R, SAS and some Business Intelligence Tools like Tableau,JMP.\nAfter the completion of the course I am looking forward to have knowledge of all the concepts of Business Analytics.\n\nMaster of IT in Business\n\nStrategy & Organisation\nText Analytics\nCloud and Big Data Analytics\nData Analytics Lab\nAnalytics Framework and Business Context\nSpreadSheet Modelling\nFinancial Accounting\nManagement Accounting\nPr